# Anvil - Python Web App Demo

This notebook demonstrates how to use Anvil to quickly build a Python webapp.

[Link to Tutorial](https://anvil.works/learn/tutorials/google-colab-to-web-app)

[Anvil Documentation](https://anvil.works/docs/overview)

In [1]:
!pip install anvil-uplink -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00


In [2]:
import anvil.server

## Functions for Chatbot

In [3]:
import torch
from huggingface_hub import login
login()

In [18]:
# Download model
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# model_id = "meta-llama/Llama-3.2-1B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = 'arputtick/llama_3.2_1B_Haraway'
# model_id = 'arputtick/llama_3.2_1B_EcoFem'
# model_id = 'arputtick/llama_3.2_1B_EcoFem_checkpoint200'#
model_id = 'arputtick/llama_3.2_1B_EcoFem_v2'

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    temperature = 0.9, # Higher temperature make less probable tokens more likely
    # do_sample = False, # If false, should make model deterministic.
    top_p = 0.95, # The most probable words adding up to total of 95% will be sampled.
    # repetition_penalty = 1.1
)

Device set to use cuda:0


In [23]:
def generate_text(chat_history, user, system):
    # Generating text
    messages = [
    {"role": "system", "content": system },
    {"role": "user", "content": user},
    ]
    chat_history.extend(messages)
    print("\n\nGenerating text...")
    outputs = pipe(
        chat_history,
        max_new_tokens=500,
        do_sample=True
    )
    response = outputs[0]["generated_text"][-1]['content']
    # chat_history.append({"role": "assistant", "content": response})
    return response

## Anvil API - Link Notebook to Webapp

Webapp URL: https://etbsujjm5yn3viid.anvil.app/DYMIQMQFTRBCQ6XPPRKOSSDX

In [24]:
anvil.server.connect('server_Y45X2BOKPFK4TRGVPH5ULV6J-ETBSUJJM5YN3VIID')

In [27]:
@anvil.server.callable
def get_response(chat_history, user, system):
    return generate_text(chat_history, user, system)